# Credit Card Recommender

This notebook will demo the credit card recommendation workflow utilizing GPT. 

## Preprocessing
1. Read in transaction history (CSV) -> convert to JSON string
2. Read credit card data as JSON

In [32]:
import numpy as np
import pandas as pd
from pprint import pprint
import json

from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
# Import transaction data
tx_df = pd.read_csv('tx-data.csv')
tx_df.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No


In [5]:
tx_json = tx_df.to_dict(orient="records")

In [6]:
pprint(tx_json)

[{'Amount': '$134.09',
  'Card': 0,
  'Day': 1,
  'Errors?': nan,
  'Is Fraud?': 'No',
  'MCC': 5300,
  'Merchant City': 'La Verne',
  'Merchant Name': 3527213246127876953,
  'Merchant State': 'CA',
  'Month': 9,
  'Time': '06:21',
  'Use Chip': 'Swipe Transaction',
  'User': 0,
  'Year': 2002,
  'Zip': 91750.0},
 {'Amount': '$38.48',
  'Card': 0,
  'Day': 1,
  'Errors?': nan,
  'Is Fraud?': 'No',
  'MCC': 5411,
  'Merchant City': 'Monterey Park',
  'Merchant Name': -727612092139916043,
  'Merchant State': 'CA',
  'Month': 9,
  'Time': '06:42',
  'Use Chip': 'Swipe Transaction',
  'User': 0,
  'Year': 2002,
  'Zip': 91754.0},
 {'Amount': '$120.34',
  'Card': 0,
  'Day': 2,
  'Errors?': nan,
  'Is Fraud?': 'No',
  'MCC': 5411,
  'Merchant City': 'Monterey Park',
  'Merchant Name': -727612092139916043,
  'Merchant State': 'CA',
  'Month': 9,
  'Time': '06:22',
  'Use Chip': 'Swipe Transaction',
  'User': 0,
  'Year': 2002,
  'Zip': 91754.0},
 {'Amount': '$128.95',
  'Card': 0,
  'Day': 2

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [29]:
# Import credit card data
with open("credit-card-data.json", "rb") as file:
    credit_card_json = json.load(file)
pprint(credit_card_json)

{'creditCards': [{'APR': '20.99% - 27.99%',
                  'annualFee': 95,
                  'benefits': ['$300 in travel credits in the first year',
                               'Primary rental car insurance',
                               'No foreign transaction fees',
                               '$150 in additional partnership benefit value',
                               '1:1 point transfer with partners',
                               'Travel and purchase coverage'],
                  'cardName': 'Chase Sapphire Preferred Card',
                  'cardType': 'Credit Card',
                  'countryOfOrigin': 'USA',
                  'creditCardScoreMax': 850,
                  'creditCardScoreMin': 650,
                  'issuer': 'Chase',
                  'linkToApply': 'https://creditcards.chase.com/rewards-credit-cards/sapphire/preferred',
                  'rewards': {'pointsPerDollar': {'dining': 3,
                                                  'onlineGrocer

## Connect to OpenAI API

In [8]:
client = OpenAI()

pprint(client.models.list().data)

[Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system'),
 Model(id='babbage-002', created=1692634615, object='model', owned_by='system'),
 Model(id='davinci-002', created=1692634301, object='model', owned_by='system'),
 Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
 Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system'),
 Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai

### Let's try feeding both JSONs directly to the model

### Quick demo

In [23]:
llm = ChatOpenAI(model="chatgpt-4o-latest")
output_parser = StrOutputParser()

In [57]:
rec_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a world-class expert in credit cards and you specialize in making credit card recommendations."),
    (
        "user", """
        Here is the user's transaction history:
        ```json
        {transactions}
        ```
        
        Here is the user's annual income: $<income>
        
        Here is our database of credit card information:
        ```json
        {credit_cards}
        ```
        
        Please make a recommendation on the credit cards that provide the most value based on the user's transaction history.
        
        Format your answer as follows:
        1. Think through the problem step-by-step. Talk about possible factors to consider when choosing a credit card as a customer
        2. Taking all these factors into account, explain your reasoning step-by-step, which ultimately leads to your recommendations. Ensure to explain your reasoning for why you ranked some cards higher/lower than others.
        3. Rank your recommendations in ascending order by most recommended to least recommended. Output your recommendations in a list along with their value proposition like this:
        
        <answer>
        1. <card name>: <reasoning>
        2. <card name>: <reasoning>
        ...
        </answer>
        """)
])

In [58]:
rec_chain = rec_prompt | llm | output_parser

In [59]:
travel_info = tx_df.query("MCC == 4722")
travel_info["Transaction Amount"] = np.random.uniform(5000, 100000, travel_info.shape[0])

income = 50000000

output = rec_chain.invoke({
    "transactions": travel_info.to_dict(orient="records"),
    "income": income,
    "credit_cards": credit_card_json,
})

/var/folders/yk/cgh45y4d46z76c3w_cj1qv3w0000gn/T/ipykernel_79554/4209876240.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  travel_info["Transaction Amount"] = np.random.uniform(5000, 100000, travel_info.shape[0])


In [60]:
import re
from rich.console import Console
from rich.markdown import Markdown

sections = re.split(r"###", output)

console = Console()

for section in sections:
    if section.strip():
        markdown_content = Markdown(f"### {section.strip()}")
        console.print(markdown_content)

Step 1: Think through the problem step-by-step                                   

When recommending a credit card based on transaction history, there are several factors to consider:               

 1 Spending Categories:                                                                                            
   The user’s transaction history indicates a pattern of online spending, with transactions frequently associated  
   with MCC 4722, which is categorized under travel agencies and tour operators. As such, a card that rewards      
   travel and online purchases would be beneficial.                                                                
 2 Transaction Frequency and Refunds:                                                                              
   The user frequently engages in both purchases and refunds. This suggests that a card with good purchase         
   protection or flexible refund policies would be valuable.                                                       
 3 Annual Fee:                                                                                                     
   Depending on the user's income (which you have not provided), the annual fee tolerance may vary. High annual fee
   cards usually offer premium benefits, but this may be overkill if the user’s income is moderate or if they do   
   not take full advantage of the perks.                                                                           
 4 APR:                                                                                                            
   If the user carries a balance, then APR might be an important factor. However, the transaction history doesn’t  
   indicate whether the user carries a balance or pays in full. Therefore, I will prioritize rewards and benefits  
   over APR, but mention low APR options for balance carriers.                                                     
 5 Credit Score:                                                                                                   
   Since the user’s credit score is not provided, I will assume they have a good score (likely 690+ based on their 
   transaction patterns and the cards they currently use). This opens up the possibility of premium cards.         
 6 Sign-Up Bonuses:                                                                                                
   If the user has significant upcoming expenses (e.g., travel), a card with a strong sign-up bonus could offer    
   considerable value if they can meet the spending requirements.                                                  
 7 Additional Benefits:                                                                                            
   Travel protections, rental car coverage, no foreign transaction fees, and other perks are valuable for a user   
   who frequently engages in travel-related purchases.

Step 2: Reasoning and Analysis                                           

Given the user's heavy online spending (especially in travel categories) and the historical mix of high-value and  
medium-value transactions, the most suitable cards will prioritize travel rewards, online purchase rewards, and    
general flexibility (e.g., no foreign transaction fees, purchase protection). Cards with significant sign-up       
bonuses and travel protections will be prioritized.

Card-by-Card Analysis:                                               

 1 Chase Sapphire Preferred Card:                                                                                  
    • Value Proposition: This card offers 5 points per dollar for travel and 3 points for dining and online grocery
      purchases, making it a strong fit for the user’s transaction history (which involves a lot of travel). The   
      60,000-point sign-up bonus after spending $4,000 in the first 3 months is also attractive. Additional perks  
      like no foreign transaction fees, primary rental car insurance, and a generous travel credit make it an      
      excellent choice for someone who frequently engages in travel-related transactions. The annual fee is $95,   
      which is reasonable given the benefits.                                                                      
    • Why Recommended: This card aligns perfectly with the user’s spending on travel and online purchases. The     
      sign-up bonus and travel-related perks add significant value.                                                
 2 Capital One Venture Rewards Credit Card:                                                                        
    • Value Proposition: This card offers 5 points per dollar on travel-related categories such as hotels, rental  
      cars, and vacation rentals, which matches the user’s frequent travel-related transactions. The 75,000-mile   
      sign-up bonus after spending $4,000 in the first 3 months is very generous. Benefits like no foreign         
      transaction fees, auto rental coverage, and extended warranty are also useful for a frequent traveler. The   
      annual fee is $95, which provides a good trade-off for the benefits.                                         
    • Why Recommended: The user’s travel-heavy spending aligns with the rewards structure, and the sign-up bonus is
      appealing for immediate benefits.                                                                            
 3 Chase Sapphire Reserve:                                                                                         
    • Value Proposition: This premium card offers 5 points per dollar on travel and 10 points on dining. It comes  
      with premium benefits like a $300 annual travel credit, airport lounge access, and travel insurance, making  
      it an excellent choice for frequent travelers. However, it has a steep annual fee of $550. While the user’s  
      transaction history suggests significant travel-related spending, it’s unclear whether they would fully      
      utilize the premium benefits, such as lounge access.                                                         
    • Why Recommended: This card’s premium travel benefits are unmatched, but the high annual fee makes it less    
      attractive unless the user can fully take advantage of its perks.                                            
 4 AmEx Platinum Card:                                                                                             
    • Value Proposition: This is another premium card with a high annual fee of $695. It offers 5 points per dollar
      on travel and a host of luxury benefits, including airport lounge access, a $200 airline fee credit, and     
      various travel protections. While these benefits are valuable, the user would need to fully take advantage of
      the perks to justify the high annual fee.                                                                    
    • Why Recommended: The card offers premium travel benefits but ranks lower due to its high annual fee. It’s    
      only suitable if the user travels frequently and can make use of the luxury perks.                           
 5 Citi Double Cash Card:                                                                                          
    • Value Proposition: This card offers a straightforward 2% cash back on all purchases (1% when yo

Step 3: Recommendations

Ranked Recommendations:                                              

 1 Chase Sapphire Preferred Card: Offers excellent rewards in travel and dining, which align with the user's       
   spending habits. The sign-up bonus and travel protections make it the best all-around option.                   
 2 Capital One Venture Rewards Credit Card: A strong contender with great travel rewards and a high sign-up bonus. 
   Slightly less flexible than Chase Sapphire Preferred but still a great option for travel-heavy users.           
 3 Chase Sapphire Reserve: Premium travel benefits, but the high annual fee only makes sense if the user can fully 
   leverage perks like lounge access and the $300 travel credit.                                                   
 4 AmEx Platinum Card: Luxury travel benefits, but the annual fee is very high. Better suited for frequent luxury  
   travelers.                                                                                                      
 5 Citi Double Cash Card: A good all-purpose card with 2% cash back and no annual fee, but lacks the travel rewards
   the user could benefit from.                                                                                    
 6 Discover it Cash Back: No annual fee and rotating 5% categories, but misaligned with the user’s travel-heavy    
   spending.                                                                                                       
 7 Wells Fargo Reflect Card: Best for balance transfers and long 0% APR periods, but not useful for someone who    
   could earn travel rewards.